## 情感分布考察

In [9]:
import os
import pandas as pd
import numpy as np
import paddlehub as hub

In [10]:
df = pd.read_csv("xue_distance_final.csv")

In [11]:
df = df['content']

In [12]:
df

0                                              要黑胶的私我 免费七天
1                                                     cpdd
2                                               桥牌的谎言是什么意思
3                                   “你永远不知道 我假装不在乎的样子有多难受”
4        在这么冷的天?想抽根电子?烟可瑞克没有电可是雪豹已失联摘不下我虚伪的假面几句胡言被奉为圣箴尝...
                               ...                        
85260                                             我在广东没看过雪
85261                                                   第一
85262                                      我愿把胸膛解剖 公开脆弱的结构
85263                                               啊啊啊啊啊啊
85264                                                  第一！
Name: content, Length: 85265, dtype: object

In [ ]:
# 加载模型
senta = hub.Module(name="senta_bilstm")

Download https://bj.bcebos.com/paddlehub/paddlehub_dev/senta_bilstm_1.2.0.tar.gz
[######################                            ] 45.85%